In [ ]:
import datetime as dt
import getpass
import os as os
import time

# Ruta de la carpeta
carpeta = "datos_HCHO"

# Verificar si existe
if not os.path.exists(carpeta):
    raise FileNotFoundError(f"La carpeta '{carpeta}' no existe.")

# Listar los archivos
archivos = os.listdir(carpeta)

# Borrar cada archivo dentro de la carpeta
for f in archivos:
    ruta_archivo = os.path.join(carpeta, f)
    if os.path.isfile(ruta_archivo):
            os.remove(ruta_archivo)

            print("✅ Todos los archivos dentro de la carpeta han sido borrados.")


import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from xarray.plot.utils import label_from_attrs

from harmony import BBox, Client, Collection, Request
from harmony.config import Environment
                    
username = "pedro.arrivas"
password = "pEdrito02@123"


harmony_client = Client(env=Environment.PROD, auth=(username, password))


print(f"Logging in as {username}")



In [ ]:
instante_actual = time.localtime()

annio = int(instante_actual.tm_year)
mes = int(instante_actual.tm_mon)
dia = int(instante_actual.tm_mday)
hora = int(instante_actual.tm_hour)
minuto = int(instante_actual.tm_min)

print(f"Fecha y hora actual: {annio}-{mes}-{dia} {hora}:{minuto}")

request = Request(
    collection=Collection(id="C3685912035-LARC_CLOUD"),
    # Note there is not a granule specified!
    temporal={
        "start": dt.datetime(annio, mes, dia, hora-1, minuto, 0),
        "stop": dt.datetime(annio, mes, dia, hora, minuto, 0),
    },
    variables=["geolocation/latitude","geolocation/longitude","product/vertical_column"],
)


job_id = harmony_client.submit(request)

print(f"jobID = {job_id}")
harmony_client.wait_for_processing(job_id, show_progress=True)





In [ ]:
# Download the resulting files
results = harmony_client.download_all(job_id, directory="datos_HCHO", overwrite=True)

all_results_stored = [f.result() for f in results]
print(f"Number of result files: {len(all_results_stored)}")

In [8]:
import requests
import datetime as dt
from harmony import Client, Collection, Request
from harmony.config import Environment

CMR_BASE = "https://cmr.earthdata.nasa.gov/search/granules.json"

def get_latest_granule_for_bbox(collection_concept_id: str, bbox: tuple) -> dict:
    # bbox = (min_lon, min_lat, max_lon, max_lat)
    bbox_str = ",".join(map(str, bbox))
    params = {
        "collection_concept_id": collection_concept_id,
        "bounding_box": bbox_str,
        "page_size": 1,
        "sort_key": "-start_date"  # pedir el más reciente (CMR soporta sort_key)
    }
    r = requests.get(CMR_BASE, params=params, timeout=30)
    r.raise_for_status()
    results = r.json()
    items = results.get("feed", {}).get("entry", [])
    if not items:
        return None
    return items[0]  # metadatos del granule más reciente

def request_with_harmony(username, password, collection_id, temporal, bbox, variables, out_dir):
    client = Client(env=Environment.PROD, auth=(username, password))
    req = Request(
        collection=Collection(id=collection_id),
        temporal={"start": temporal[0], "stop": temporal[1]},
        spatial=bbox,                # algunas versiones usan 'bbox' o 'spatial' — revisar tu harmony-py
        variables=variables,
    )
    job_id = client.submit(req)
    client.wait_for_processing(job_id, show_progress=True)
    results = client.download_all(job_id, directory=out_dir, overwrite=True)
    return [f.result() for f in results]

In [9]:
# Usar la función request_with_harmony para extraer los datos en la carpeta 'datos_HCHO'
# Utilizamos las variables ya definidas en el notebook

collection_id = "C3685912035-LARC_CLOUD"
temporal = (
    dt.datetime(annio, mes, dia, hora-1, minuto, 0),
    dt.datetime(annio, mes, dia, hora, minuto, 0)
)
bbox = None  # Si quieres limitar por área, define una tupla (min_lon, min_lat, max_lon, max_lat)
variables = ["geolocation/latitude", "geolocation/longitude", "product/vertical_column"]
out_dir = carpeta

# Si quieres usar bbox, reemplaza None por una tupla de coordenadas
result_files = request_with_harmony(
    username, password, collection_id, temporal, bbox, variables, out_dir
)

print(f"Archivos descargados: {result_files}")

Exception: ('Bad Request', 'Error: No matching granules found.')

In [ ]:
import os
import xarray as xr

# Carpeta con tus archivos .nc
carpeta = "datos_HCHO"

# Lista de archivos .nc
archivos_nc = sorted([os.path.join(carpeta, f) for f in os.listdir(carpeta) if f.endswith(".nc")])

# Verifica que haya archivos
if not archivos_nc:
    raise FileNotFoundError("No se encontraron archivos .nc en la carpeta especificada.")




In [ ]:

import h5netcdf
import pandas as pd
import numpy as np


# Crear un diccionario para almacenar los DataTree o Datasets
datatree_dict = {}
df_total=pd.DataFrame()
for archivo in archivos_nc:
    print(f"\n📂 Leyendo archivo: {archivo}")

    # Intentar abrir como Dataset (más común)
    try:
        ds = xr.open_dataset(archivo)
    except Exception:
        # Si no funciona, abrir como DataTree
        ds = xr.open_datatree(archivo)

    datatree_dict[archivo] = ds

    # Mostrar rango de tiempo si existe la variable geolocation/time
    if 'geolocation/time' in ds.coords:
        print(f"🕓 Rango de tiempo: {ds['geolocation/time'].values.min()} - {ds['time'].values.max()}")

    with h5netcdf.File(archivo, 'r') as f:
        lat = f['geolocation/latitude'][:]
        lon = f['geolocation/longitude'][:]
        time = f['geolocation/time'][:]
        hcho = f['product/vertical_column'][:]

        # Leer los atributos del tiempo
        time_attrs = f['geolocation/time'].attrs
        time_units = time_attrs.get('units', '')
        calendar = time_attrs.get('calendar', 'standard')

        print(f"🕓 Atributos de tiempo: units={time_units}, calendar={calendar}")

    # Asegurar que lat, lon, hcho tengan misma forma
    if hcho.shape != lat.shape:
        min_shape = tuple(np.minimum(lat.shape, hcho.shape))
        lat = lat[:min_shape[0], :min_shape[1]]
        lon = lon[:min_shape[0], :min_shape[1]]
        hcho = hcho[:min_shape[0], :min_shape[1]]

    # Expandir el tiempo a toda la grilla
    if len(time.shape) == 1:
        if len(time) == lat.shape[0]:
            time_expand = np.repeat(time[:, np.newaxis], lat.shape[1], axis=1)
        elif len(time) == 1:
            time_expand = np.full_like(lat, time[0], dtype=float)
        else:
            time_expand = np.full_like(lat, np.mean(time), dtype=float)
    else:
        time_expand = time

    # Convertir el tiempo correctamente usando las unidades del archivo
    try:
        if "since" in time_units:
            times_dt = xr.coding.times.decode_cf_datetime(time_expand, units=time_units, calendar=calendar)
        else:
            # Si no hay unidades, asumimos segundos desde 1970
            times_dt = pd.to_datetime(time_expand, unit='s', errors='coerce')
    except Exception as e:
        print(f"⚠️ No se pudo convertir el tiempo: {e}")
        times_dt = pd.to_datetime(time_expand, unit='s', errors='coerce')

    # Crear DataFrame
    df = pd.DataFrame({
        'latitud': lat.flatten(),
        'longitud': lon.flatten(),
        'tiempo': times_dt.flatten(),
        'HCHO_molecules_per_cm2': hcho.flatten()
    })

    # Filtrar datos inválidos
    df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=['HCHO_molecules_per_cm2'])

    # Agregar al total
    df_total = pd.concat([df_total, df], ignore_index=True)

print("\n✅ Lectura completa de todos los archivos .nc")

print(df_total.head())




In [ ]:
max_hcho = df_total['HCHO_molecules_per_cm2'].max()
min_hcho = df_total['HCHO_molecules_per_cm2'].min()
print(f"🔺 Máximo HCHO: {max_hcho:.3e} molecules/cm²")
print(f"🔻 Mínimo HCHO: {min_hcho:.3e} molecules/cm²")

In [ ]:
import pandas as pd

def obtener_hcho_por_coordenada(df, latitud, longitud, tolerancia=0.5):
    """
    Devuelve los valores de formaldehído (HCHO) en todas las fechas disponibles
    para una ubicación específica (latitud, longitud), ordenados del más reciente al más antiguo.
    
    Parámetros:
        df (pd.DataFrame): DataFrame con columnas ['latitud', 'longitud', 'tiempo', 'HCHO_molecules_per_cm2']
        latitud (float): latitud a consultar
        longitud (float): longitud a consultar
        tolerancia (float): margen de búsqueda en grados (default = 0.05)
    
    Retorna:
        pd.DataFrame ordenado por tiempo (más reciente primero)
    """

    # Filtrar por cercanía a la coordenada dada
    filtro = (
        (df["latitud"].between(latitud - tolerancia, latitud + tolerancia)) &
        (df["longitud"].between(longitud - tolerancia, longitud + tolerancia))
    )
    
    resultados = df[filtro].copy()
    
    if resultados.empty:
        print("⚠️ No se encontraron datos cercanos a las coordenadas dadas.")
        return None

    # Convertir tiempo a formato datetime (en segundos desde época o ISO)
    resultados["tiempo"] = pd.to_datetime(resultados["tiempo"], errors="coerce", unit="s")

    # Ordenar por tiempo (más reciente primero)
    resultados = resultados.sort_values(by="tiempo", ascending=False)

    print(f"📍 Coordenadas consultadas: ({latitud}, {longitud}) ± {tolerancia}°")
    print(f"🕓 Rango temporal: {resultados['tiempo'].min()} → {resultados['tiempo'].max()}")
    print(f"🔢 Registros encontrados: {len(resultados)}")

    return resultados[["tiempo", "latitud", "longitud", "HCHO_molecules_per_cm2"]]


In [ ]:
df_total.sort_values(by="HCHO_molecules_per_cm2", ascending=False).head()


In [ ]:

resultados = obtener_hcho_por_coordenada(df_total, latitud=22, longitud=-67)
print(resultados.head())



In [ ]:
import pandas as pd

def obtener_hcho_reciente_por_coordenada(df, latitud, longitud, tolerancia=0.5):
    """
    Devuelve el valor más reciente de formaldehído (HCHO) para una ubicación específica
    (latitud, longitud), excluyendo valores negativos y ordenando por tiempo descendente.
    
    Parámetros:
        df (pd.DataFrame): DataFrame con columnas ['latitud', 'longitud', 'tiempo', 'HCHO_molecules_per_cm2']
        latitud (float): latitud a consultar
        longitud (float): longitud a consultar
        tolerancia (float): margen de búsqueda en grados (default = 0.5)
    
    Retorna:
        pd.DataFrame con una sola fila (el valor más reciente) o None si no hay datos válidos.
    """

    # Filtrar por cercanía a la coordenada dada
    filtro = (
        (df["latitud"].between(latitud - tolerancia, latitud + tolerancia)) &
        (df["longitud"].between(longitud - tolerancia, longitud + tolerancia))
    )

    resultados = df[filtro].copy()

    # Eliminar valores negativos o no válidos
    resultados = resultados[resultados["HCHO_molecules_per_cm2"] > 0]

    if resultados.empty:
        print("⚠️ No se encontraron datos válidos (positivos) cercanos a las coordenadas dadas.")
        return None

    # Asegurar formato datetime
    resultados["tiempo"] = pd.to_datetime(resultados["tiempo"], errors="coerce")

    # Ordenar por tiempo descendente
    resultados = resultados.sort_values(by="tiempo", ascending=False)

    # Tomar el valor más reciente
    reciente = resultados.iloc[0]

    print(f"📍 Coordenadas consultadas: ({latitud}, {longitud}) ± {tolerancia}°")
    print(f"🕓 Fecha más reciente: {reciente['tiempo']}")
    print(f"🧪 HCHO más reciente: {reciente['HCHO_molecules_per_cm2']:.3e} molecules/cm²")

    return pd.DataFrame([reciente])[["tiempo", "latitud", "longitud", "HCHO_molecules_per_cm2"]]


In [ ]:
resultado = obtener_hcho_reciente_por_coordenada(df_total, latitud=22, longitud=-67)
print(resultado)
